<a href="https://colab.research.google.com/github/Sirius-Aabhas/CS69002_9A_18CS60R55/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data_utils
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device('cuda:0')
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


device(type='cuda', index=0)

##Fetching data

In [3]:
from google.colab import files
import io
uploaded = files.upload()

Saving Train_20K.csv to Train_20K (1).csv


In [4]:
type(uploaded), uploaded.keys(), type(uploaded['Train_20K.csv'])


(dict, dict_keys(['Train_20K.csv']), bytes)

In [5]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
df.head()

,text,label
0,John Waters has given us a genuinely enjoyable...,1
1,This first two seasons of this comedy series w...,1
2,"What an unfortunate mess is ""Shiner."" I wanted...",0
3,I'm not entirely sure Rob Schmidt qualifies as...,1
4,i wasn't sure whether to laugh or cry. Porrett...,0


In [6]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [7]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]

filtered_text_reviews = []
for sent in text_reviews:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews = filtered_text_reviews
print(text_reviews[0], text_labels[0])
print(len(text_reviews), len(text_labels))

john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments 1
17999 17999


In [8]:
data = [(text_reviews[i], text_labels[i])for i in range(len(text_labels))]
data[:5]

[('john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments',
  1),
 ('first two seasons comedy series strange werent funny drama element bill mother struggling usual problems life element bit depressing didnt mix well th comedy elements probably dropped soon became one funniest comedy series bbc ever made chemistry bill bens characters funny always many brilliant memorable sketches series christmas specials hilarious real treat christmas show came stop main actor gary olsen playing bill passed away sad brilliant actor films n funny man ripbr underrated show sadly disappeared television screens doesnt repeated often though appear uktv gold repeated bbc one two show brilliant comedy new audience',
  1),
 ('unfortunate mess shiner wanted like overthetop antifilm aspirant fact f

In [12]:
from google.colab import files
import io
uploaded = files.upload()

Saving Test_5K.csv to Test_5K (1).csv


In [13]:
type(uploaded), uploaded.keys(), type(uploaded['Test_5K.csv'])

(dict, dict_keys(['Test_5K.csv']), bytes)

In [14]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
df_test.head()

,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [15]:
print('Number of Negative movie reviews', len(df_test[df_test['label']==0]))
print('Number of Positive movie reviews', len(df_test[df_test['label']==1]))

Number of Negative movie reviews 2482
Number of Positive movie reviews 2518


In [16]:
text_reviews_test = df_test['text'].astype(str).tolist()
text_labels_test = df_test['label'].astype(int)

text_reviews_test = [x.lower() for x in text_reviews_test]

filtered_text_reviews = []
for sent in text_reviews_test:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews_test = filtered_text_reviews

print(text_reviews_test[0], text_labels_test[0])
print(len(text_reviews_test), len(text_labels_test))

always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner shame

In [37]:
data_test = [(text_reviews_test[i], text_labels_test[i])for i in range(len(text_labels_test))]
data_test[:5]

[('always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner sh

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
    word_to_ix = {}
    word_cntr = {}
    word_set = set()
    #print(dataset)
    for sent in dataset:
        for word in sent.split():
            if word not in word_cntr:
                word_cntr[word] = 1
            else:
                word_cntr[word] += 1
    
    for word in word_cntr:
        if word_cntr[word] >= 5:
            word_to_ix[word] = len(word_to_ix)
            
    word_to_ix['<UNKNOWN>'] = len(word_to_ix)
        
    return word_to_ix

In [28]:
word_to_ix = generate_word_ids(text_reviews + text_reviews_test)
VOCAB_SIZE = len(word_to_ix)
VOCAB_SIZE

30624

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['<UNKNOWN>']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [43]:
t1 = time.time()
features = make_bow_vector(text_reviews[0], word_to_ix).to(device)
for i in range(1,len(text_reviews)):
    vec = make_bow_vector(text_reviews[i], word_to_ix).to(device)
    features = torch.cat((features,vec)).to(device)
    if i%1000 == 0:
        print(time.time() - t1)
        t1 = time.time()
        print(features.shape)
targets = torch.tensor(text_labels, dtype=torch.int).to(device)

print(time.time() - t1)
print(features.shape)
print(targets.shape)
train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=32, shuffle=True)

2.1611812114715576
torch.Size([1001, 30624])
3.379796266555786
torch.Size([2001, 30624])
5.205514192581177
torch.Size([3001, 30624])
7.2094032764434814
torch.Size([4001, 30624])
9.266945838928223
torch.Size([5001, 30624])
11.305914163589478
torch.Size([6001, 30624])
13.67388653755188
torch.Size([7001, 30624])
15.828752517700195
torch.Size([8001, 30624])
17.930289030075073
torch.Size([9001, 30624])
20.040346145629883
torch.Size([10001, 30624])
22.143686056137085
torch.Size([11001, 30624])
24.16520047187805
torch.Size([12001, 30624])
121.0351734161377
torch.Size([13001, 30624])
170.08469438552856
torch.Size([14001, 30624])
181.17917203903198
torch.Size([15001, 30624])
193.8370544910431
torch.Size([16001, 30624])
208.7408492565155
torch.Size([17001, 30624])
220.95192861557007
torch.Size([17999, 30624])
torch.Size([17999])


In [44]:
features[0].shape

torch.Size([30624])

##Task1

In [0]:
class Task1A(nn.Module):
    def __init__(self, vocab_size):
        super(Task1A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Task1B(nn.Module):
    def __init__(self, vocab_size):
        super(Task1B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Task1C(nn.Module):
    def __init__(self, vocab_size):
        super(Task1C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Task1D(nn.Module):
    def __init__(self, vocab_size):
        super(Task1D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 500)
        self.lin2 = nn.Linear(500, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
model1A = Task1A(VOCAB_SIZE)
model1B = Task1B(VOCAB_SIZE)
model1C = Task1C(VOCAB_SIZE)
model1D = Task1D(VOCAB_SIZE)

In [0]:
for param in model1A.parameters():
    print(param,param.size())

##Task2

In [0]:
class Task2A(nn.Module):
    def __init__(self, vocab_size):
        super(Task2A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 10)
        self.lin2 = nn.Linear(10, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Task2B(nn.Module):
    def __init__(self, vocab_size):
        super(Task2B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 20)
        self.lin2 = nn.Linear(20, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Task2C(nn.Module):
    def __init__(self, vocab_size):
        super(Task2C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 30)
        self.lin3 = nn.Linear(30, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Task2D(nn.Module):
    def __init__(self, vocab_size):
        super(Task2D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Task2E(nn.Module):
    def __init__(self, vocab_size):
        super(Task2E, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model2A = Task2A(VOCAB_SIZE)
model2B = Task2B(VOCAB_SIZE)
model2C = Task2C(VOCAB_SIZE)
model2D = Task2D(VOCAB_SIZE)
model2E = Task2E(VOCAB_SIZE)

##Task3

In [0]:
class Task3A(nn.Module):
    def __init__(self, vocab_size):
        super(Task3A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Task3B(nn.Module):
    def __init__(self, vocab_size):
        super(Task3B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 100)
        self.lin3 = nn.Linear(100, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
model3A = Task3A(VOCAB_SIZE)
model3B = Task3B(VOCAB_SIZE)

##Task4

In [0]:
class Task4A(nn.Module):
    def __init__(self, vocab_size):
        super(Task4A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.drop_layer(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Task4B(nn.Module):
    def __init__(self, vocab_size):
        super(Task4B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 100)
        self.lin3 = nn.Linear(100,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Task4C(nn.Module):
    def __init__(self, vocab_size):
        super(Task4C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 10)
        self.lin3 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model4A = Task4A(VOCAB_SIZE)
model4B = Task4B(VOCAB_SIZE)
model4C = Task4C(VOCAB_SIZE)

##Task5

In [0]:
class Task5A(nn.Module):
    def __init__(self, vocab_size):
        super(Task5A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Task5B(nn.Module):
    def __init__(self, vocab_size):
        super(Task5B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.tanh(self.lin1(x))
        x = F.tanh(self.lin2(x))
        x = self.drop_layer(x)
        x = F.tanh(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Task5C(nn.Module):
    def __init__(self, vocab_size):
        super(Task5C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.sigmoid(self.lin1(x))
        x = F.sigmoid(self.lin2(x))
        x = self.drop_layer(x)
        x = F.sigmoid(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model5A = Task5A(VOCAB_SIZE)
model5B = Task5B(VOCAB_SIZE)
model5C = Task5C(VOCAB_SIZE)

In [0]:
for param in model5A.parameters():
    print(param,param.size())

##Task7

In [0]:
class Task7A(nn.Module):
    def __init__(self, vocab_size):
        super(Task7A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Task7B(nn.Module):
    def __init__(self, vocab_size):
        super(Task7B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Task7C(nn.Module):
    def __init__(self, vocab_size):
        super(Task7C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model7A = Task7A(VOCAB_SIZE)
model7B = Task7B(VOCAB_SIZE)
model7C = Task7C(VOCAB_SIZE)

##Train, validate, save

In [0]:
def make_one_hot(labels):
    ret_list = torch.zeros((len(labels), 2))
    for i, label in enumerate(labels):
        ret_list[i][label] = 1
    return ret_list

In [0]:
def train(model, isGpu):
    if isGpu:
        model = model.to(device)
    # define a loss function and an optimizer
    #loss_function = nn.NLLLoss()
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr = 0.1)
    # the training loop
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
def train_MSE_Hinge(model, isGpu, loss_function):
    if isGpu:
        model = model.to(device)
        
    opt = torch.optim.SGD(model.parameters(), lr = 0.1)
    # the training loop
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            label = make_one_hot(label).to(device)
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
def validate(model):
    model = model.cpu()
    preds = []
    for instance, label in data_test:
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        logprobs = model(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        preds.append(pred)

    print('accuracy: {}'.format(accuracy_score(text_labels_test, preds)))
    print('precision: {}'.format(precision_score(text_labels_test, preds)))
    print('recall: {}'.format(recall_score(text_labels_test, preds)))
    print('f1-score: {}'.format(f1_score(text_labels_test, preds)))


In [0]:
def run_and_save(model): 
    train(model, True)
    validate(model)
    torch.save(model, model._get_name()+'.mdl')

In [0]:
run_and_save(model1B)
run_and_save(model1C)
run_and_save(model1D)
run_and_save(model2A)
run_and_save(model2B)
run_and_save(model2C)
run_and_save(model2D)
run_and_save(model2E)
run_and_save(model3A)
run_and_save(model3B)
run_and_save(model4A)
run_and_save(model4B)
run_and_save(model4C)
run_and_save(model5A)
run_and_save(model5B)
run_and_save(model5C)
run_and_save(model7A)
train_MSE_Hinge(model7B, True, nn.MSELoss())
validate(model7B)
torch.save(model7B, 'Task7B.mdl')
train_MSE_Hinge(model7C, True, nn.HingeEmbeddingLoss())
validate(model7C)
torch.save(model7C, 'Task7C.mdl')

In [0]:
mdl_1a = torch.load('Task7B.mdl')

In [0]:
validate(mdl_1a)

##Task6


###Bigram generation

In [0]:
bigrm_list = []
for text in text_reviews+text_reviews_test:
    words = text.split()
    bigrm_list += (b for b in zip(words[:-1], words[1:]))

In [0]:
bigrm_cntr = {}
bigrm_to_ix = {}
for bigrm in bigrm_list:
    if bigrm not in bigrm_cntr:
        bigrm_cntr[bigrm] = 1
    else:
        bigrm_cntr[bigrm] += 1

for bigrm in bigrm_cntr:
    if bigrm_cntr[bigrm] >= 5:
        bigrm_to_ix[bigrm] = len(bigrm_to_ix)
        
bigrm_to_ix['<UNKNOWN>'] = len(bigrm_to_ix)

In [71]:
BIGRAM_VOCAB_SIZE = len(bigrm_to_ix)
BIGRAM_VOCAB_SIZE

57224

###Model

In [0]:
class Task6(nn.Module):
    def __init__(self, vocab_size):
        super(Task6, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.3)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))
    
model6 = Task6(BIGRAM_VOCAB_SIZE)

###Trainig, validation and save

In [0]:
def make_bigrm_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in zip(sentence.split()[:-1], words[1:]):
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['<UNKNOWN>']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [74]:
model6 = model6.to(device)
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model6.parameters(), lr = 0.5)
for epoch in range(1):
    tic = time.time()
    for instance, label in data:
        model6.zero_grad()
        vec = Variable(make_bigrm_bow_vector(instance, bigrm_to_ix)).to(device)
        label = Variable(make_target(label)).to(device)
        probs = model6(vec) # forward pass
        loss = loss_function(probs, label)
        loss.backward()
        opt.step()
    toc = time.time()
    print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


EPOCH: 0, CURRENT LOSS: 0.31326162815093994, TIME TAKEN: 54.89679265022278


In [95]:
model6 = model6.cpu()
preds = []
for instance, label in data_test:
    bow_vec = Variable(make_bigrm_bow_vector(instance, bigrm_to_ix))
    logprobs = model6(bow_vec)
    #print(logprobs)
    pred = np.argmax(logprobs.data.cpu().numpy())
    preds.append(pred)

print('accuracy: {}'.format(accuracy_score(text_labels_test, preds)))
print('precision: {}'.format(precision_score(text_labels_test, preds)))
print('recall: {}'.format(recall_score(text_labels_test, preds)))
print('f1-score: {}'.format(f1_score(text_labels_test, preds)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


accuracy: 0.4964
precision: 0.0
recall: 0.0
f1-score: 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [96]:
torch.save(model6, 'Task6.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task6. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
import pickle
with open('bigram_dictionary.pkl', 'wb') as f:
    pickle.dump(bigrm_to_ix, f, pickle.HIGHEST_PROTOCOL)

##Task8

In [0]:
class Task8(nn.Module):
    def __init__(self, vocab_size):
        super(Task8, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.drop_layer(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))
    
model8 = Task8(BIGRAM_VOCAB_SIZE)

In [0]:
model8 = model8.to(device)
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model6.parameters(), lr = 0.05)
patience = 4
best_loss = 0.
for epoch in range(20):
    tic = time.time()
    for instance, label in data:
        model8.zero_grad()
        vec = Variable(make_bow_vector(instance, bigrm_to_ix)).to(device)
        label = Variable(make_target(label)).to(device)
        probs = model8(vec) # forward pass
        loss = loss_function(probs, label)
        loss.backward()
        opt.step()
    toc = time.time()
    print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))
    
    if loss.data < best_loss:
        best_loss = loss.data
        patience = 4
    else:
        patience -=1 
        if patience == 0:
            break


In [0]:
def train_MSE_Hinge(model, isGpu, loss_function):
    if isGpu:
        model = model.to(device)
        
    opt = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
    # the training loop
    patience = 5
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            label = make_one_hot(label).to(device)
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))
        if epoch == 0:
            best_loss = loss.data
            continue
        if loss.data < best_loss:
            best_loss = loss.data
            patience = 5
        else:
            patience -=1 
            if patience == 0:
                break


In [115]:
train_MSE_Hinge(model7C, True, nn.HingeEmbeddingLoss())
validate(model7C)
torch.save(model7C, 'Task7C.mdl')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


EPOCH: 0, CURRENT LOSS: 0.6000000238418579, TIME TAKEN: 2.444589853286743
EPOCH: 1, CURRENT LOSS: 0.7000000476837158, TIME TAKEN: 2.2382571697235107
in else
EPOCH: 2, CURRENT LOSS: 0.6333333849906921, TIME TAKEN: 2.226335287094116
in else
EPOCH: 3, CURRENT LOSS: 0.6333333849906921, TIME TAKEN: 2.230898380279541
in else
EPOCH: 4, CURRENT LOSS: 0.6666666865348816, TIME TAKEN: 2.2563865184783936
in else
EPOCH: 5, CURRENT LOSS: 0.6333333849906921, TIME TAKEN: 2.2417232990264893
in else
accuracy: 0.3378
precision: 0.11012782694198624
recall: 0.04447974583002383
f1-score: 0.06336633663366337


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task7C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
